In [1]:
from mod import CONFIG, process_ano, get_ano_info, coor_check, read_anomalies, play_ano
from general import PATHS
import os
import cv2

In [2]:
vid_no =33
mode = "train"

In [3]:
play_ano(vid_no, mode, process_ano(vid_no, mode))

Number of anomalies:  4
Anomaly 1
(2, [[300, 102, 335, 122], [62, 2], [22463, 749]])
Anomaly 2
(206, [[641, 32, 656, 45], [2018, 67], [24423, 814]])
Anomaly 3
(782, [[583, 154, 614, 180], [4820, 161], [26819, 894]])
Anomaly 4
(2586, [[335, 54, 355, 68], [15660, 522], [26819, 894]])


In [4]:
# sorted_ano = process_ano(vid_no, "train")
# print("Number of anomalies: ", len(lmao3.keys()) )
# for i, ano_id in enumerate(lmao3.items()):
#     print(f"Anomaly {i+1}")
#     print(ano_id)

In [5]:
# def display_ano(vid_no, ano_data, mode): 
    coors, start, end = ano_data

    if mode == "train":
        file_path = PATHS["dataset"] + f'\\aic21-track4-train-data\\{vid_no}.mp4'
    elif mode == "test":
        file_path = PATHS["dataset"] + f'\\aic21-track4-test-data\\{vid_no}.mp4'
    else:
        file_path = None

    try:
        cap = cv2.VideoCapture(file_path)
    except:
        print("#####################\nICORRECT MODE\n**************************")

    x1, x2, y1, y2 = coors
    
    # Check if camera opened successfully
    if (cap.isOpened()== False): 
        print("Error opening video stream or file")
    fps = 30
    # Set the starting frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, start[0])
    # Read until video is completed
    while(cap.isOpened()):
    # Capture frame-by-frame
        ret, frame = cap.read()

        current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
        
        if current_frame > end[0]:
            break

        timer = int(round(current_frame/fps, 0))

        if ret == True:
            # Display the resulting frame
            frame = cv2.rectangle(frame, 
                                pt1=(x1 - CONFIG['anomaly_range'], x2 - CONFIG['anomaly_range']), 
                                pt2= (y1 + CONFIG['anomaly_range'], y2 + CONFIG['anomaly_range']),  
                                color= (0, 0, 255), thickness= 2)
            
            frame = cv2.putText(frame, f'time: {timer}', 
                          org= (20, 50), 
                          fontFace= cv2.FONT_HERSHEY_COMPLEX,
                          fontScale= 1, 
                          color= (0, 255, 0), 
                          thickness= 1)

            cv2.imshow(f'frame{vid_no}',frame)

            # Press Q on keyboard to  exit
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
        
        # Break the loop
        # else: 
        #   break
    cap.release()

    # Closes all the frames
    cv2.destroyAllWindows()


IndentationError: unexpected indent (3312899692.py, line 2)

In [ ]:
# print("Number of anomalies: ", len(sorted_ano.keys()) )
# for i, ano_id in enumerate(sorted_ano.items()):
#     print(f"Anomaly {i+1}")
#     print(ano_id)
#     display_ano(vid_no, get_ano_info(sorted_ano, ano_id[0]), mode)

Number of anomalies:  4
Anomaly 1
(2, [[300, 102, 335, 122], [62, 2], [22463, 749]])
Anomaly 2
(206, [[641, 32, 656, 45], [2018, 67], [24423, 814]])
Anomaly 3
(782, [[583, 154, 614, 180], [4820, 161], [26819, 894]])
Anomaly 4
(2586, [[335, 54, 355, 68], [15660, 522], [26819, 894]])


In [ ]:
# train_2 = read_anomalies(vid_no, "train")
# train_2

In [ ]:
# filtered_data = {k: v for k, v in train_2.items() if (v[3][1] - v[2][1]) >= 20}

# def merge_similar_coordinates(data):
#     # Helper function to check if two coordinate lists are similar
#     def are_similar_coords(coords1, coords2, tolerance=5):
#         return all(abs(c1 - c2) <= tolerance for c1, c2 in zip(coords1, coords2))

#     merged_data = {}
#     visited_keys = set()

#     for key1, value1 in data.items():
#         if key1 in visited_keys:
#             continue

#         coords1, trash1, start1, end1 = value1
#         merged_start, merged_end = start1, end1

#         for key2, value2 in data.items():
#             if key2 <= key1 or key2 in visited_keys:
#                 continue

#             coords2, trash2, start2, end2 = value2
#             if are_similar_coords(coords1, coords2):
#                 merged_start = min(merged_start, start2)
#                 merged_end = max(merged_end, end2)
#                 visited_keys.add(key2)

#         merged_data[key1] = [coords1, merged_start, merged_end]
#         visited_keys.add(key1)

#     return merged_data
# final_data = merge_similar_coordinates(filtered_data)

# print("Number of anomalies: ", len(final_data.keys()) )
# for i, ano_id in enumerate(final_data.items()):
#     print(f"Anomaly {i+1}")
#     print(ano_id)

In [ ]:
# def new_sort_ano(anomalies: dict):
    filtered_data = {k: v for k, v in anomalies.items() if (v[3][1] - v[2][1]) >= CONFIG['anomaly_min_time']}

    merged_data = {}
    visited_keys = set()

    for key1, value1 in filtered_data.items():
        if key1 in visited_keys:
            continue

        coords1, trash1, start1, end1 = value1
        merged_start, merged_end = start1, end1

        for key2, value2 in filtered_data.items():
            if key2 <= key1 or key2 in visited_keys:
                continue

            coords2, trash2, start2, end2 = value2
            if coor_check(coords1, coords2, errors= CONFIG['error'] + CONFIG['anomaly_range']):
                merged_start = min(merged_start, start2)
                merged_end = max(merged_end, end2)
                visited_keys.add(key2)

        merged_data[key1] = [coords1, merged_start, merged_end]
        visited_keys.add(key1)
    
    return merged_data

IndentationError: unexpected indent (3141023849.py, line 2)

In [ ]:
# def sort_ano(anomailes: dict):
    ano = {}
    anomailes_cop = anomailes.copy()

    for ano_id1 in anomailes.keys():
        if ano_id1 in anomailes_cop.keys():
            temp = []
            coors1 = anomailes[ano_id1][0]
            start1 = anomailes[ano_id1][-2]
            end1 = anomailes[ano_id1][-1]
            if end1[1] - start1[1] >= CONFIG['anomaly_min_time']:
                ano[ano_id1] = [coors1, start1, end1]
                for ano_id2 in anomailes_cop.keys():
                    if ano_id1 == ano_id2:
                        pass
                    else:
                        coors2 = anomailes[ano_id2][0]
                        start2 = anomailes[ano_id2][-2]
                        end2 = anomailes[ano_id2][-1]

                        if coor_check(coors1, coors2, errors= CONFIG["error"] + CONFIG['anomaly_range']):
                            ano[ano_id1] = [coors1, start1, end2]
                            temp.append(ano_id2)
            else:
                pass

            for i in temp:
                anomailes_cop.pop(i)  
    return ano

In [ ]:
# lmao2 = new_sort_ano(train_2)
# print("Number of anomalies: ", len(lmao2.keys()) )
# for i, ano_id in enumerate(lmao2.items()):
#     print(f"Anomaly {i+1}")
#     print(ano_id)

Number of anomalies:  4
Anomaly 1
(2, [[300, 102, 335, 122], [62, 2], [1931, 64]])
Anomaly 2
(206, [[641, 32, 656, 45], [2018, 67], [26819, 894]])
Anomaly 3
(782, [[583, 154, 614, 180], [4820, 161], [26819, 894]])
Anomaly 4
(2586, [[335, 54, 355, 68], [15660, 522], [26819, 894]])


In [ ]:
# lmao = sort_ano(train_2)
# print("Number of anomalies: ", len(lmao.keys()) )
# for i, ano_id in enumerate(lmao.items()):
#     print(f"Anomaly {i+1}")
#     print(ano_id)

Number of anomalies:  4
Anomaly 1
(2, [[300, 102, 335, 122], [62, 2], [2695, 90]])
Anomaly 2
(206, [[641, 32, 656, 45], [2018, 67], [24423, 814]])
Anomaly 3
(782, [[583, 154, 614, 180], [4820, 161], [26819, 894]])
Anomaly 4
(2586, [[335, 54, 355, 68], [15660, 522], [26819, 894]])
